

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/DEID_PHI_TEXT.ipynb)




# **Deidentify free text documents**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload `license_keys.json` to the folder that opens.
Otherwise, you can look at the example outputs at the bottom of the notebook.



## 1. Colab Setup

Import license keys

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

Install dependencies

In [2]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

# Installing Spark NLP Display Library for visualization
! pip install -q spark-nlp-display

Import dependencies into Python

In [3]:
import json
import os
import pandas as pd

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import warnings
warnings.filterwarnings('ignore')



Start the Spark session

In [4]:
params = {"spark.driver.memory":"16G", 
          "spark.kryoserializer.buffer.max":"2000M", 
          "spark.driver.maxResultSize":"2000M"} 

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

# manually start session
# params = {"spark.driver.memory" : "16G",
#           "spark.kryoserializer.buffer.max" : "2000M",
#           "spark.driver.maxResultSize" : "2000M"}

# spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

Spark NLP Version : 3.3.4
Spark NLP_JSL Version : 3.3.4


## 2. Select the NER model and construct the pipeline

Select the models:


* NER Deidentification models: **ner_deid_enriched, ner_deid_large**

* Deidentification models: **deidentify_large, deidentify_rb, deidentify_rb_no_regex**





For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [5]:
# Change this to the model you want to use and re-run the cells below.
# Anatomy models: ner_anatomy
MODEL_NAME = "ner_deid_large"
DEID_MODEL_NAME = "deidentify_large"

Create the pipeline

In [6]:
documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")


sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")


tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on n2c2 datasets)
clinical_ner = MedicalNerModel.pretrained(MODEL_NAME, "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"])\
    .setOutputCol("ner")

# NER Overwriter to ensure all the entities are deidentified.
# Use this if the NER does not recognize entities.
neroverwriter = NerOverwriter() \
    .setInputCols(["ner"]) \
    .setOutputCol("ner_overwrited") \
    .setStopWords(['AIQING', 'YBARRA']) \
    .setNewResult("B-NAME")

ner_converter = NerConverterInternal()\
  .setInputCols(["sentence", "token", "ner_overwrited"])\
  .setOutputCol("ner_chunk")

nlp_pipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    neroverwriter,
    ner_converter])

empty_df = spark.createDataFrame([['']]).toDF('text')
pipeline_model = nlp_pipeline.fit(empty_df)
light_pipeline = LightPipeline(pipeline_model)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_deid_large download started this may take some time.
Approximate size to download 14.1 MB
[OK!]


## 3. Create example inputs

In [7]:
# Enter examples as strings in this array
input_list = [
    """Patient AIQING, 25 month years-old , born in Beijing, was transfered to the The Johns Hopkins Hospital. Phone number: (541) 754-3010. MSW 100009632582 for his colonic polyps. He wants to know the results from them. He is not taking hydrochlorothiazide and is curious about his blood 
pressure. He said he has cut his alcohol back to 6 pack once a week. He 
has cut back his cigarettes to one time per week. P:   Follow up with Dr. Hobbs in 3 months. Gilbert P. Perez, M.D."""]

## 4. Run the pipeline to find Entities

In [8]:
result = pipeline_model.transform(spark.createDataFrame(pd.DataFrame({'text':input_list})))

Visualize

In [9]:
exploded = F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata'))
select_expression_0 = F.expr("cols['0']").alias("chunk")
select_expression_1 = F.expr("cols['1']['entity']").alias("ner_label")
result.select(exploded.alias("cols")) \
    .select(select_expression_0, select_expression_1).show(truncate=False)

+----------------------+---------+
|chunk                 |ner_label|
+----------------------+---------+
|AIQING                |NAME     |
|25                    |AGE      |
|Beijing               |LOCATION |
|Johns Hopkins Hospital|LOCATION |
|(541) 754-3010        |CONTACT  |
|100009632582          |ID       |
|Hobbs                 |NAME     |
|Gilbert P. Perez      |NAME     |
+----------------------+---------+



# 5. Deidentify using Obfuscation Method

In [10]:
obfuscation = DeIdentificationModel.pretrained(DEID_MODEL_NAME, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("obfuscated") \
      .setMode("obfuscate")

deid_text = obfuscation.transform(result)

deidentify_large download started this may take some time.
Approximate size to download 54.7 KB
[OK!]


# 6. Visualize Obfusacted Results

In [11]:
deid_text.select(F.explode(F.arrays_zip('sentence.result', 'obfuscated.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"Patient AIQING, 25 month years-old , born in B...","Patient Pearly Calix, 85 month years-old , bor..."
1,Phone number: (541) 754-3010.,Phone number: 09704 13 48 83.
2,MSW 100009632582 for his colonic polyps.,MSW E8141471 for his colonic polyps.
3,He wants to know the results from them.,He wants to know the results from them.
4,He is not taking hydrochlorothiazide and is cu...,He is not taking hydrochlorothiazide and is cu...
5,He said he has cut his alcohol back to 6 pack ...,He said he has cut his alcohol back to 6 pack ...
6,He \nhas cut back his cigarettes to one time p...,He \nhas cut back his cigarettes to one time p...
7,P: Follow up with Dr. Hobbs in 3 months.,P: Follow up with Dr. Lurena Sharp in 3 months.
8,"Gilbert P. Perez, M.D.","Serina Sinner, M.D."


## 7. Deidentify using Masking Method

In [12]:
masking = DeIdentificationModel.pretrained(DEID_MODEL_NAME, "en", "clinical/models") \
      .setInputCols(["sentence", "token", "ner_chunk"]) \
      .setOutputCol("masked") \
      .setMode("mask")

deid_text = masking.transform(result)

deidentify_large download started this may take some time.
Approximate size to download 54.7 KB
[OK!]


# 8. Visualize Masked Results

In [13]:
deid_text.select(F.explode(F.arrays_zip('sentence.result', 'masked.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("sentence"), F.expr("cols['1']").alias("deidentified")).toPandas()

,sentence,deidentified
0,"Patient AIQING, 25 month years-old , born in B...","Patient <NAME>, <AGE> month years-old , born i..."
1,Phone number: (541) 754-3010.,Phone number: <CONTACT>.
2,MSW 100009632582 for his colonic polyps.,MSW <ID> for his colonic polyps.
3,He wants to know the results from them.,He wants to know the results from them.
4,He is not taking hydrochlorothiazide and is cu...,He is not taking hydrochlorothiazide and is cu...
5,He said he has cut his alcohol back to 6 pack ...,He said he has cut his alcohol back to 6 pack ...
6,He \nhas cut back his cigarettes to one time p...,He \nhas cut back his cigarettes to one time p...
7,P: Follow up with Dr. Hobbs in 3 months.,P: Follow up with Dr. <NAME> in 3 months.
8,"Gilbert P. Perez, M.D.","<NAME>, M.D."
